In [1]:
!pip install streamlit pyngrok transformers langchain faiss-cpu PyPDF2 sentence-transformers
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successful

**IMPORT LIBRARIES**

In [2]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import pickle
import os

**1) PDF PARSING**

In [3]:
@st.cache_resource
def load_pdfs(pdf_files):
    """Parse PDFs and extract text."""
    documents = []
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        text = "".join([page.extract_text() for page in reader.pages])
        documents.append(text)
    return documents

**2) Vector Store Initialization (Using FAISS)**

In [4]:
def initialize_vector_store(pdf_texts):
    """Create embeddings and store them in FAISS for similarity search."""
    # Split text --> chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = [chunk for doc in pdf_texts for chunk in text_splitter.split_text(doc)]

    # Generate embeddings
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding_model)

    # Save vector store
    with open("vector_store.pkl", "wb") as f:
        pickle.dump(vector_store, f)

    return vector_store

**3) Retrieval QA Chain**

In [5]:
def initialize_qa_chain(_vector_store):
    """Set up the retrieval-augmented QA chain."""
    # Set up retriever
    retriever = _vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    # Configure the language model (LLM)
    llm_pipeline = pipeline(
        "text-generation",
        model="gpt2",
        tokenizer="gpt2",
        max_length=1024,
        max_new_tokens=200,
        pad_token_id=50256
    )
    llm = HuggingFacePipeline(pipeline=llm_pipeline)

    # Create the QA chain
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

**4) Streamlit Interface**

In [6]:
st.title("10-K Document QA System")
st.write("Ask questions about the financial documents (Google, Tesla, Uber).")

pdf_files = [
    "/content/drive/MyDrive/ALMENO_AI/goog-10-k-2023 (1).pdf",  # Path to Google 10-K PDF
    "/content/drive/MyDrive/ALMENO_AI/tsla-20231231-gen.pdf",   # Path to Tesla 10-K PDF
    "/content/drive/MyDrive/ALMENO_AI/uber-10-k-2023.pdf"     # Path to Uber 10-K PDF
]

# Step 2: Process PDFs or Load Vector Store
if os.path.exists("vector_store.pkl"):
    # Load  vector store
    with open("vector_store.pkl", "rb") as f:
        vector_store = pickle.load(f)
    st.write("Vector store loaded from saved data!")
else:
    # ParsING
    pdf_texts = load_pdfs(pdf_files)
    vector_store = initialize_vector_store(pdf_texts)
    st.write("Vector store created and saved successfully!")

# Step 3: Initialize QA Chain
qa_chain = initialize_qa_chain(vector_store)

# Step 4: User Query and Display Response
query = st.text_input("Enter your question:", "What are the risk factors associated with Google and Tesla?")
if st.button("Get Answer"):
    with st.spinner("Retrieving the best answer..."):
        response = qa_chain.run(query)
    st.write("### Response:")
    st.write(response)

2024-11-26 11:10:09.275 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.762 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-26 11:10:09.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:10:09.781 Thread 'MainThread': mi

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<ipython-input-5-7ecc1e912a04>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)
2024-11-26 11:16:15.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:16:15.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:16:15.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:16:15.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 11:16:15.501 Sessio

In [8]:
query =  "Total revenue of Uber?"
response = qa_chain.run(query)

Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [9]:
print(response)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

revenue, incentives and promotions as further described in Note 1 – Description of Business and Summary of Significant Accounting Policies in the notes to the
consolidated financial statements included in Part II, Item 8, “Financial Statements and Supplementary Data,” and the section titled “Critical Accounting
Estimates” in Part II, Item 7, of this Annual Report on Form 10-K.
48Financial and Operational Highlights
Year Ended December 31, Constant Currency 
(In millions, except per centages) 2021 20222021 to 2022 %
Change 2021 to 2022 % Change
Monthly Active Platform Consumers (“MAPCs”) 118 131 11 %
Trips 6,368 7,642 20 %
Gross Bookings $ 90,415 $ 115,395 28 % 33 %
Revenue $ 17,455 $ 31,877 83 % 90 %
Net loss attributable to Uber Technologies, Inc. $ (496)$ (9,141) **
Mobility Adjusted EBITDA $ 1,596 $ 3,299 107 %
Delivery A

**Code for extracting relevant answer part**

In [10]:
# Extract the Helpful Answer part from the response
start = response.find("Helpful Answer:")
if start != -1:
    # Extract everything after "Helpful Answer:"
    helpful_answer = response[start + len("Helpful Answer:"):].strip()
    print(helpful_answer)
else:
    print("Helpful Answer not found.")

In 2021, the valuation of the transaction will be based on the amount of revenue for the period immediately prior to the commencement of the transaction. Additionally, revenue will be recognized in the transaction's initial period end user discount, which in turn

represents any of the following services received:

- a delivery that we would expect to deliver to end users in December in the first five months of a calendar year if the price/demand level were stable

- an app that users might use to reach end users in the first five months of the previous year


***After building the content engine. We deploy it on an UI, using Streamlit.***

*We use Localtunnel to expose streamlit app online.*

In [11]:
!pip install streamlit pyngrok transformers langchain faiss-cpu PyPDF2 sentence-transformers --quiet
!npm install -g localtunnel



added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [15]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.101.41:8501

your url is: https://fancy-corners-taste.loca.lt
/content/app.py:7: LangChainDeprecationWarning: Importing HuggingFaceEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.embeddings import HuggingFaceEmbeddings
/content/app.py:8: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import FAISS

with new imports of:

>> from langchain_community.vectorstores import 

In [13]:
!wget -q -O - ipv4.icanhazip.com #To get the tunnel Passcode

35.247.101.41


In [36]:
#PUT THIS IN app.py file

import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from PyPDF2 import PdfReader
import pickle
import os

# --------------------- Helper Functions ---------------------
@st.cache_resource
def load_pdfs(pdf_paths):
    """Parse hardcoded PDFs and extract their text."""
    documents = []
    for pdf_file in pdf_paths:
        reader = PdfReader(pdf_file)
        text = "".join([page.extract_text() for page in reader.pages])
        documents.append(text)
    return documents

@st.cache_resource
def initialize_vector_store(pdf_texts):
    """Create embeddings and store them in FAISS for similarity search."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = [chunk for doc in pdf_texts for chunk in text_splitter.split_text(doc)]

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding_model)

    with open("vector_store.pkl", "wb") as f:
        pickle.dump(vector_store, f)

    return vector_store

@st.cache_resource
def initialize_qa_chain(_vector_store):
    """Set up the retrieval-augmented QA chain."""
    retriever = _vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    llm_pipeline = pipeline(
        "text-generation",
        model="gpt2",
        tokenizer="gpt2",
        max_length=1024,
        max_new_tokens=200,
        pad_token_id=50256
    )
    llm = HuggingFacePipeline(pipeline=llm_pipeline)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

# --------------------- Streamlit Interface ---------------------
st.title("10-K Document QA System")
st.write("Ask questions about the financial documents (Google, Tesla, Uber).")

# Step 1: Hardcoded PDF Paths
pdf_files = [
    "google_10k.pdf",  # Path to Google 10-K PDF
    "tesla_10k.pdf",   # Path to Tesla 10-K PDF
    "uber_10k.pdf"     # Path to Uber 10-K PDF
]

# Step 2: Process PDFs or Load Precomputed Vector Store
if os.path.exists("vector_store.pkl"):
    with open("vector_store.pkl", "rb") as f:
        vector_store = pickle.load(f)
    st.write("Vector store loaded from saved data!")
else:
    pdf_texts = load_pdfs(pdf_files)
    vector_store = initialize_vector_store(pdf_texts)
    st.write("Vector store created and saved successfully!")

# Step 3: Initialize QA Chain
qa_chain = initialize_qa_chain(vector_store)

# Step 4: User Query
query = st.text_input("Enter your question:", "What are the risk factors associated with Google and Tesla?")
if st.button("Get Answer"):
    with st.spinner("Retrieving the best answer..."):
        response = qa_chain.run(query)
    st.write("### Response:")
    # st.write(response)

    # Extract the Helpful Answer part from the response
    start = response.find("Helpful Answer:")  # Find the start of the Helpful Answer section
    if start != -1:
        # Extract everything after "Helpful Answer:"
        helpful_answer = response[start + len("Helpful Answer:"):].strip()
        # Print only the helpful answer text
        st.write(helpful_answer)
    else:
        print("Helpful Answer not found.")


2024-11-26 06:33:41.391 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.399 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-26 06:33:41.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar